In [37]:
import sys
import os.path
import importlib.util

# insert path to the local version of spacer_tutorial package into pythonpath
if os.path.exists("src/spacer_tutorial/"):
    sys.path.append(os.path.abspath("src"))
    print("Using local version of spacer_tutorial package")

# install z3 if needed
if importlib.util.find_spec('z3') is None:
    !{sys.executable} -m pip install z3-solver
# install spacer_tutorial if needed
if importlib.util.find_spec('spacer_tutorial') is None:
    !{sys.executable} -m pip install spacer-tutorial

Using local version of spacer_tutorial package


In [6]:
from z3 import *
from spacer_tutorial import *

# proof mode must be enabled before any expressions are created
z3.set_param(proof=True)
z3.set_param(model=True)
# print expressions with HTML
z3.set_html_mode(True)

# Experiments with Scaling

## Utilities

In [3]:
# Binary Craig Interpolation by reduction to CHC
def interpolate(A, B, verbosity=0):
    '''Computes a Craig interpolant between A and B'''
    As = free_arith_vars(A)
    Bs = free_arith_vars(B)

    shared = [s for s in As & Bs ]

    Itp = z3.Function('Itp', [s.sort() for s in shared] + [z3.BoolSort()])
    left = z3.ForAll([a for a in As], z3.Implies(A, Itp(shared)))
    right = z3.ForAll([b for b in Bs], z3.Implies(Itp(shared), z3.Not(B)))

    res, answer = solve_horn([left, right])

    if res == z3.sat:
        return answer.eval(Itp(shared))
    return None 

In [4]:
def sub(formula, constants):
    subed = formula
    # print(z3.substitute(scaled,[(z3.IntVal(2), z3.Int('K_N'))]))
    for c in constants:
        k_c = f'K_{c}'
        subed = z3.substitute(subed, (z3.IntVal(c), z3.Int(k_c)))
    return subed

In [5]:
def scale(formula, constants, K, k=0):
    scaled = formula
    for c in constants:
        # k_s = 'K'
        scaled = z3.substitute(scaled, (z3.IntVal(c), c*K))
    # scaled = And(scaled, K > k)
    return scaled

## src

In [6]:
# scale(2*x + 3*y + 4 <= 0 && 3*y + 2 <= 0, k) = 2*x + 3*y + 4k <= 0 && 3y + 2k <= 0
T = Solver()
x, x_out = Ints("x0 x1")
y, y_out = Ints("y0 y1")
K = Int('K')

A = And(y > 4)
B = And(3*x + 3*y + 4 <= 0, 3*y + 2 <= 0)
# Init = x == 0
# Tr = Or(And(x < 5, x_out == x + 1), And(x >= 453, x_out == x))
# Bad = And(x >= 6)
T.add(A)
T.add(B)
T

[y0 &gt; 4, 3&middot;x0 + 3&middot;y0 + 4 &le; 0 &and; 3&middot;y0 + 2 &le; 0]

In [7]:
res = T.check()
display(res)

if res == sat:
    display(T.model())
else:
    display(interpolate(A, B))

<b>unsat</b>

&not;(y0 &le; 4)

## subed

In [8]:
if res == unsat:
    B2 = sub(B,[2, 3, 4])
    display(B2)

T2 = Solver()
T2.add(A)
T2.add(B2)
resSubed = T2.check()
display(resSubed)

if resSubed == sat:
    display(T2.model())
else:
    display(interpolate(A, B2))

K_3&middot;x0 + K_3&middot;y0 + K_4 &le; 0 &and; K_3&middot;y0 + K_2 &le; 0

<b>sat</b>

[K_4 = 0, K_2 = 0, y0 = 5, K_3 = 0, x0 = 0]

## scaled

In [9]:
B3 = B
# Var = z3.Int('K')
if res == unsat:
    A3 = And(scale(A,list([2, 4]), K))
    B3 = And(scale(B,list([2, 4]), K), K > 0)
    display(A3)
    display(B3)
assert(not B3.__str__().__eq__(B.__str__()))

# T3 = Optimize()
T3 = Solver()
T3.add(A3)
T3.add(B3)
# T3.add_soft(z3.Int('K') >= 1)
display(T3)

resScaled = T3.check()
# resScaled = T3.check(z3.Int('K') >= 1)
display(resScaled)

if resScaled == sat:
    display(T3.model())
else:
    display(interpolate(A3, B3))

y0 &gt; 4&middot;K

3&middot;x0 + 3&middot;y0 + 4&middot;K &le; 0 &and; 3&middot;y0 + 2&middot;K &le; 0 &and; K &gt; 0

[y0 &gt; 4&middot;K, 3&middot;x0 + 3&middot;y0 + 4&middot;K &le; 0 &and; 3&middot;y0 + 2&middot;K &le; 0 &and; K &gt; 0]

<b>unsat</b>

&not;(4&middot;K + -1&middot;y0 &ge; 0)

# Experiments with IC3

In [10]:
T = Solver()
x, x_out = Ints("x0 x1")

Init = x == 0
Tr = Or(And(x < 5, x_out == x + 1), And(x >= 453, x_out == x))
Bad = And(x >= 6)

In [11]:
T = Solver()
F2 =[]
T.add(*F2)
T.add(Bad)
T.check()
T.model()

[x0 = 6]

In [12]:
# pob = x_out == T.model().eval(x)
pob = z3.substitute(Bad, (x, x_out))

# pob = [x >= 5, x >= 6]
pob

x1 &ge; 6

In [13]:
S = Solver()
F1 = [Not(And(x >= 6))]
S.add(*F1)
S.add(Tr)
S.add(pob)
res = S.check()
if res == sat:
    display(S.model())
else:
    display(res)


<b>unsat</b>

In [14]:

pob_1 = z3.substitute(z3.And(x >= 6), (x, x_out))
pob_1

x1 &ge; 6

In [15]:
S1 = Solver()
F0 = [Init]
S1.add(*F0)
S1.add(Tr)
S1.add(pob_1)
res = S1.check()
if res == sat:
    display(S.model())
else:
    display(res)

<b>unsat</b>

In [16]:

proxy, proxy2 = z3.Bools('proxy proxy2')

In [17]:
s = z3.Solver()
s.add(z3.Implies(proxy, z3.And(x > 0, x < 2)))
s.add(z3.Implies(proxy2, z3.And(x_out > 0, x_out < 2)))
a = z3.Bool('a')
s.add(z3.Implies(a, x > 1))
s

[proxy &rArr; x0 &gt; 0 &and; x0 &lt; 2,
 proxy2 &rArr; x1 &gt; 0 &and; x1 &lt; 2,
 a &rArr; x0 &gt; 1]

In [18]:
s.check(proxy, z3.Not(proxy2), a)

<b>unsat</b>

In [19]:
s.check(proxy2, a)

<b>sat</b>

# Experiments with New SMT Solver

## Setup

In [1]:
def vc_gen(T):
    '''Verification Condition (VC) for an Inductive Invariant'''
    Inv = z3.Function('Inv', *(T.sig() + [z3.BoolSort()]))

    InvPre = Inv(*T.pre_vars())
    InvPost = Inv(*T.post_vars())
    print ('pres', T.pre_vars())
    print ('post', T.post_vars())
    print ('inputs', T.inputs())

    all_vars = T.all()
    vc_init = z3.ForAll(all_vars, z3.Implies(T.Init, InvPre))
    vc_ind = z3.ForAll(all_vars, z3.Implies(z3.And(InvPre, T.Tr), InvPost))
    vc_bad = z3.ForAll(all_vars, z3.Implies(z3.And(InvPre, T.Bad), z3.BoolVal(False)))
    return [vc_init, vc_ind, vc_bad], InvPre

challenge1 = False

def mk_ts_challenge(safe=True):
    T = Ts('Ts0')
    constants = list()
    x, x_out = T.add_var(z3.IntSort(), name='x')
    y, y_out = T.add_var(z3.IntSort(), name='y')
    if challenge1:
        # Challenge 1
        constants = list([5000, 10000])
        T.Init = z3.And(x == 0, y == 5000)
        T.Tr = z3.And(x_out == x + 1, y_out == z3.If(x >= 5000, y + 1, y))
        if safe:
            T.Bad = z3.And(x == 10000, z3.Not(y == x))
        else:
            T.Bad = z3.Not(z3.And(x == 1000, z3.Not(y == x)))
    else:
        # Challenge 2
        constants = list([1351235, 2702470])
        T.Init = z3.And(x == 0, y == 0)
        T.Tr = z3.And(x_out == x + 1, y_out == z3.If(x % 2 == 0, y + 1, y))
        if safe:
            T.Bad = z3.And(x % 2 == 0, z3.Not(2*y == x))
            # T.Bad = z3.And(x == z3.Product(2, z3.IntVal(1351235)), z3.Not(y == 1351235))
        else:
            T.Bad = z3.Not(z3.And(x == z3.Product(2, 1351235), z3.Not(y == 1351235)))
    return T, constants

## Normal

In [26]:
ts_challenge, constants = mk_ts_challenge(safe=True)
HtmlStr(ts_challenge)
vc_challenge, inv_challenge = vc_gen(ts_challenge)
chc_to_str(vc_challenge)
display(chc_to_str(vc_challenge))
res_challenge, answer_challenge = solve_horn(vc_challenge, max_unfold=40)
display (res_challenge)
if res_challenge == sat:
    push_not(answer_challenge.eval(inv_challenge))

pres [x, y]
post [x', y']
inputs []


"&forall;x, y, x', y' : x = 0 &and; y = 0 &rArr; Inv(x, y) <br/><br/>&forall;x, y, x', y' :<br/> Inv(x, y) &and; x' = x + 1 &and; y' = If(x%2 = 0, y + 1, y) &rArr;<br/> Inv(x', y') <br/><br/>&forall;x, y, x', y' :<br/> Inv(x, y) &and; x = 2&middot;1351235 &and; &not;(y = 1351235) &rArr; False <br/><br/>"

<b>unknown</b>

In [71]:
def isBlocked(frames, bound, pob, solver):
    assert(bound >= 0)
    solver.push()
    solver.add(pob)
    solver.add(frames[bound])
    display(f'solver w/ POB and F[{bound}]', solver)
    check = solver.check()
    display(check)
    solver.pop()
    return check == unsat

In [84]:
def ic3_block(frames, ts, bound, pob, solver):
    assert(bound >= 0)
    solver.push()
    solver.add(frames[bound-1])
    solver.add(ts)
    solver.add(pob)
    display(f'solver w/ F[{bound-1}] and TR and POB', solver)
    check = solver.check()
    if check == unsat:
        frames[bound] = z3.Not(pob)
        display(frames)
    solver.pop()
    return check == unsat, frames

## Level 0: Substitute with Symbolic Value

In [87]:
print('entering level 0')
Tsub, constants_sub =  mk_ts_challenge(safe=True)

F = list([z3.And(Tsub.Init), z3.And(True)])
N = 1

BAD_sub = sub(Tsub.Bad, constants_sub)

IC3 = Solver()
# block_bad = ic3_block(frames=F, ts=Tsub.Tr, bound=N, pob=Tsub.Bad, solver=IC3)
# if (block_bad == unsat):
#     print(F)
if (isBlocked(frames=F, bound=N, pob=BAD_sub, solver=IC3)):
# if (isBlocked(frames=F, bound=N, pob=Tsub.Bad, solver=IC3)):
    print('Gucci')
else:
    blocked, fPrime = ic3_block(frames=F, ts=Tsub.Tr, bound=N, pob=BAD_sub, solver=IC3)
    # blocked, fPrime = ic3_block(frames=F, ts=Tsub.Tr, bound=N, pob=Tsub.Bad, solver=IC3)
    print(blocked)
    display(fPrime[N])
display('IC3: ',IC3)

# IC3.add(Tsub.Init)

# Tsub.Init = sub(Tsub.Init, constants_sub)
# Tsub.Tr = sub(Tsub.Tr, constants_sub)
# Tsub.Bad = sub(Tsub.Bad, constants_sub)
# vc_challenge_subed, inv_challenge_subed = vc_gen(Tsub)
# display(chc_to_str(vc_challenge_subed))
# res_challenge_subed, answer_challenge_subed = solve_horn(vc_challenge_subed, max_unfold=4)
# display(res_challenge_subed)
# if res_challenge_subed == sat:
#     push_not(answer_challenge_subed.eval(inv_challenge_subed))

# Exists n: x = 2n && !(y = n)
# Exists n: n = x/2 && !(y = x/2)
# x%2 = 0 && !(2*y = x)

entering level 0


'solver w/ POB and F[1]'

[x = 2&middot;K_1351235 &and; &not;(y = K_1351235), True]

<b>sat</b>

'solver w/ F[0] and TR and POB'

...

[x = 0 &and; y = 0, &not;(x = 2&middot;K_1351235 &and; &not;(y = K_1351235))]

True


&not;(x = 2&middot;K_1351235 &and; &not;(y = K_1351235))

'IC3: '

[]

## Level 1: Scale with k > 0

In [24]:
print('entering level 1')
Tscaled, constants_scaled = mk_ts_challenge(safe=True)
Tscaled.Init = scale(Tscaled.Init, constants_scaled, K)
Tscaled.Tr = scale(Tscaled.Tr,constants_scaled, K)
Tscaled.Bad = scale(Tscaled.Bad, constants_scaled, K)
vc_challenge_scaled, inv_challenge_scaled = vc_gen(Tscaled)
display(chc_to_str(vc_challenge_scaled))
res_challenge_scaled, answer_challenge_scaled = solve_horn(vc_challenge_scaled, max_unfold=40)
display(res_challenge_scaled)
if res_challenge_scaled == sat:
    push_not(answer_challenge_scaled.eval(inv_challenge_scaled))

entering level 1
pres [x, y]
post [x', y']
inputs []


"&forall;x, y, x', y' : x = 0 &and; y = 0 &rArr; Inv(x, y) <br/><br/>&forall;x, y, x', y' :<br/> Inv(x, y) &and; x' = x + 1 &and; y' = If(x%2 = 0, y + 1, y) &rArr;<br/> Inv(x', y') <br/><br/>&forall;x, y, x', y' :<br/> Inv(x, y) &and; x = 2&middot;1351235&middot;K &and; &not;(y = 1351235&middot;K) &rArr; False <br/><br/>"

<b>unknown</b>

# Equality Constraints Paper

In [38]:
def vc_gen(T):
    '''Verification Condition (VC) for an Inductive Invariant'''
    Inv = z3.Function('Inv', *(T.sig() + [z3.BoolSort()]))

    InvPre = Inv(*T.pre_vars())
    InvPost = Inv(*T.post_vars())
    print ('pres', T.pre_vars())
    print ('post', T.post_vars())
    print ('inputs', T.inputs())

    all_vars = T.all()
    vc_init = z3.ForAll(all_vars, z3.Implies(T.Init, InvPre))
    vc_ind = z3.ForAll(all_vars, z3.Implies(z3.And(InvPre, T.Tr), InvPost))
    vc_bad = z3.ForAll(all_vars, z3.Implies(z3.And(InvPre, T.Bad), z3.BoolVal(False)))
    return [vc_init, vc_ind, vc_bad], InvPre

challenge = 3

def mk_ts_challenge(safe=True):
    T = Ts('Ts0')
    constants = list()
    x, x_out = T.add_var(z3.IntSort(), name='x')
    y, y_out = T.add_var(z3.IntSort(), name='y')
    z, z_out = T.add_var(z3.IntSort(), name='z')
    if challenge == 1:
        # Challenge 1
        constants = list([5000, 10000])
        T.Init = z3.And(x == 0, y == 5000)
        T.Tr = z3.And(x_out == x + 1, y_out == z3.If(x >= 5000, y + 1, y))
        if safe:
            T.Bad = z3.And(x == 10000, z3.Not(y == x))
        else:
            T.Bad = z3.Not(z3.And(x == 1000, z3.Not(y == x)))
    elif challenge == 2:
        # Challenge 2
        constants = list([1351235, 2702470])
        T.Init = z3.And(x == 0, y == 0)
        T.Tr = z3.And(x_out == x + 1, y_out == z3.If(x % 2 == 0, y + 1, y))
        if safe:
            T.Bad = z3.And(x % 2 == 0, z3.Not(2*y == x))
            # T.Bad = z3.And(x == z3.Product(2, z3.IntVal(1351235)), z3.Not(y == 1351235))
        else:
            T.Bad = z3.Not(z3.And(x == z3.Product(2, 1351235), z3.Not(y == 1351235)))
    else:
        # Paper Challenge
        constants = list([500, 1000])
        T.Init = z3.And(x == 0, y == 0, z == 0)
        T.Tr = z3.And(x < 500, x_out == x + 1, y_out == y + x, z_out == z + x + 2)
        if safe:
            T.Bad = z3.And(x >= 500, z3.Not(z + x >= y + 1000))
        else:
            T.Bad = z3.And(x >= 500, z >= y + 1000)
    return T, constants

In [39]:
ts_challenge, constants = mk_ts_challenge(safe=True)
HtmlStr(ts_challenge)
vc_challenge, inv_challenge = vc_gen(ts_challenge)
chc_to_str(vc_challenge)
display(chc_to_str(vc_challenge))
res_challenge, answer_challenge = solve_horn(vc_challenge, max_unfold=10)
display (res_challenge)
if res_challenge == sat:
    display(push_not(answer_challenge.eval(inv_challenge)))

pres [x, y, z]
post [x', y', z']
inputs []


"&forall;x, y, z, x', y', z' : x = 0 &and; y = 0 &and; z = 0 &rArr; Inv(x, y, z) <br/><br/>&forall;x, y, z, x', y', z' :<br/> Inv(x, y, z) &and;<br/> x &lt; 500 &and;<br/> x' = x + 1 &and;<br/> y' = y + x &and;<br/> z' = z + x + 2 &rArr;<br/> Inv(x', y', z') <br/><br/>&forall;x, y, z, x', y', z' :<br/> Inv(x, y, z) &and; x &ge; 500 &and; &not;(z + x &ge; y + 1000) &rArr; False <br/><br/>"

<b>sat</b>

10 &ge; 2&middot;x + -1&middot;z + y

In [40]:
x, x_out = z3.Ints('x x_out')
y, y_out = z3.Ints('y y_out')
z, z_out = z3.Ints('z z_out')

# INV
inv1 = z3.And(10 >= 2*x - z + y)
display(inv1)

Init = z3.And(x == 0, y == 0, z == 0)
Transition = z3.And(x < 500, x_out == x + 1, y_out == y + x, z_out == z + x + 2)
Bad = z3.And(x >= 500, z3.Not(z + x >= y + 1000))
InvariantPost = z3.substitute(inv1, [(x, x_out), (y, y_out), (z, z_out)])

display(InvariantPost)

sFirst = z3.Solver()
sFirst.add(z3.And(Init, z3.Not(inv1)))
display('check first: ', sFirst)
assert sFirst.check() == z3.unsat

sSecond = z3.Solver()
sSecond.add(z3.And(inv1, Transition, z3.Not(InvariantPost)))
display('check sec: ', sSecond)
assert sSecond.check() == z3.unsat

sThird= z3.Solver()
sThird.add(z3.And(inv1, Bad))
display('check third: ', sThird)
assert sThird.check() == z3.unsat

2&middot;x - z + y &le; 10

2&middot;x_out - z_out + y_out &le; 10

'check first: '

[x = 0 &and; y = 0 &and; z = 0 &and; &not;(2&middot;x - z + y &le; 10)]

'check sec: '

[2&middot;x - z + y &le; 10 &and;
 x &lt; 500 &and;
 x_out = x + 1 &and;
 y_out = y + x &and;
 z_out = z + x + 2 &and;
 &not;(2&middot;x_out - z_out + y_out &le; 10)]

'check third: '

[2&middot;x - z + y &le; 10 &and; x &ge; 500 &and; &not;(z + x &ge; y + 1000)]